In [1]:
from urllib.request import urlretrieve
from pathlib import Path
from zipfile import ZipFile

### Data Acquisition

In [2]:
#Identify pattern of the url associated with each month's data file and code accordingly.
url_front = "https://s3.amazonaws.com/tripdata/JC-"
url_end = "-citibike-tripdata.csv.zip"

filename_base = Path('/Users/xuan/Documents/Python/portfolio_projects/ds_citibike')

In [3]:
def get_year_month_files(list_of_year_month):
    '''
    Main function to retrieve files and store them locally.
    '''
    for year_month in list_of_year_month:
        url = url_front + year_month + url_end
        filename = year_month + '.zip'
        filepath = filename_base / filename
        path, headers = urlretrieve(url, filepath)

In [4]:
#Create a function that helps with generating the year-month list needed in the main function.
def combine_year_month(year, list_of_months):
    '''
    Generates combined strings of year and month.
    Parameters are a string value of 4-digit year and a list with 2-digit, string month numbers.
    '''
    list_str_year_month = []
    for i in list_of_months:
        str_year_month = year + i
        list_str_year_month.append(str_year_month)
    return list_str_year_month        

In [5]:
#Include the relevant months.
list_of_2022_months = ['08', '09', '10', '11', '12']
list_of_2023_months = ['01', '02', '03', '04', '05', '06', '07']

In [6]:
#Call the function to get all year-month in one place.
all_year_month = combine_year_month('2022', list_of_2022_months) + combine_year_month('2023', list_of_2023_months)
print(all_year_month)

['202208', '202209', '202210', '202211', '202212', '202301', '202302', '202303', '202304', '202305', '202306', '202307']


In [7]:
#Call the main function to get all files.
get_year_month_files(all_year_month)

In [8]:
#Unzip the files
def unzip_files(list_of_year_month):
    '''
    Extract corresponding csv files for each year and month.
    '''
    for year_month in list_of_year_month:
        filename = year_month + '.zip'
        filepath = filename_base / filename
        with ZipFile(filepath, 'r') as f:
            list_of_names = f.namelist()
            for name in list_of_names:
                if name.endswith('.csv'):
                    f.extract(name)

In [9]:
unzip_files(all_year_month)